In [23]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string
import torch.nn as nn

In [24]:
# returns the pathnames matching the given path
def findFiles(path): return glob.glob(path)

In [25]:
print(findFiles('data/names/*.txt'))

['data/names\\Arabic.txt', 'data/names\\Chinese.txt', 'data/names\\Czech.txt', 'data/names\\Dutch.txt', 'data/names\\English.txt', 'data/names\\French.txt', 'data/names\\German.txt', 'data/names\\Greek.txt', 'data/names\\Irish.txt', 'data/names\\Italian.txt', 'data/names\\Japanese.txt', 'data/names\\Korean.txt', 'data/names\\Polish.txt', 'data/names\\Portuguese.txt', 'data/names\\Russian.txt', 'data/names\\Scottish.txt', 'data/names\\Spanish.txt', 'data/names\\Vietnamese.txt']


In [26]:
# all the alphabetics and the symbols
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

In [27]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [28]:
# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

In [29]:
# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

In [30]:
for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

In [31]:
all_categories

['Arabic',
 'Chinese',
 'Czech',
 'Dutch',
 'English',
 'French',
 'German',
 'Greek',
 'Irish',
 'Italian',
 'Japanese',
 'Korean',
 'Polish',
 'Portuguese',
 'Russian',
 'Scottish',
 'Spanish',
 'Vietnamese']

In [32]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [33]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('A'))
print("######")
print(lineToTensor('Abdo').shape)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
######
torch.Size([4, 1, 57])


In [34]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [35]:
input = letterToTensor('A')
hidden =torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

In [36]:
output

tensor([[-2.9751, -2.8652, -2.9189, -2.8038, -2.8812, -2.9256, -2.8533, -2.8868,
         -2.9403, -2.8007, -2.8964, -2.9214, -2.8363, -2.9294, -2.9072, -2.8531,
         -2.9725, -2.8812]], grad_fn=<LogSoftmaxBackward>)

In [37]:
next_hidden

tensor([[ 0.0679, -0.0135, -0.0322, -0.0722, -0.0471, -0.0233, -0.0679,  0.0858,
         -0.1238,  0.0728, -0.0393, -0.0018,  0.1312,  0.0771,  0.0743, -0.0196,
          0.0634,  0.1123,  0.0547,  0.0162, -0.0434,  0.0706,  0.0717,  0.0175,
         -0.0467,  0.1278,  0.0918,  0.0074,  0.0053,  0.1215,  0.0390, -0.0135,
          0.0379,  0.1246, -0.0177, -0.0116, -0.0256, -0.0047,  0.0048,  0.0688,
         -0.0076, -0.0258, -0.0062, -0.0755,  0.0630, -0.0186,  0.0476, -0.0749,
         -0.0622,  0.0217, -0.0084,  0.0184, -0.0460, -0.0061,  0.0234, -0.0062,
          0.0642, -0.1080,  0.0320, -0.0381, -0.0326,  0.0812, -0.0688, -0.0174,
         -0.1097, -0.0845, -0.0886, -0.0549,  0.0157,  0.0172, -0.1383,  0.0167,
         -0.0350, -0.0128,  0.0070,  0.0291,  0.0496,  0.0519,  0.0416, -0.0368,
         -0.0551,  0.0846,  0.0263, -0.0458, -0.0399,  0.0116, -0.0251,  0.0737,
          0.0129,  0.0945, -0.0348, -0.1120,  0.0247,  0.0772, -0.1356,  0.0551,
         -0.0074,  0.0339, -

In [38]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('Italian', 9)


In [39]:
output


tensor([[-2.9751, -2.8652, -2.9189, -2.8038, -2.8812, -2.9256, -2.8533, -2.8868,
         -2.9403, -2.8007, -2.8964, -2.9214, -2.8363, -2.9294, -2.9072, -2.8531,
         -2.9725, -2.8812]], grad_fn=<LogSoftmaxBackward>)

In [48]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = Russian / line = Piterskih
category = Greek / line = Close
category = Vietnamese / line = Luc
category = English / line = Willson
category = Arabic / line = Bitar
category = Irish / line = Tighearnach
category = Polish / line = Gorka
category = Japanese / line = Bunya
category = Chinese / line = Mak
category = Japanese / line = Mashita


In [51]:
category_tensor = torch.tensor([all_categories.index('English')], dtype=torch.long)
category_tensor

tensor([4])

In [53]:
criterion = nn.NLLLoss() #LogSoftmax

In [55]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()
    
    # loop on the letters
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [56]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 12s) 2.1760 Yoo / Korean ✓
10000 10% (0m 24s) 1.8376 Bang / Vietnamese ✗ (Korean)
15000 15% (0m 37s) 1.1654 Serafim / Portuguese ✓
20000 20% (0m 50s) 1.5948 Henriques / Dutch ✗ (Portuguese)
25000 25% (1m 3s) 0.4329 Filipkov / Russian ✓
30000 30% (1m 17s) 2.0557 Medeiros / Greek ✗ (Portuguese)
35000 35% (1m 30s) 1.1504 Fortier / French ✓
40000 40% (1m 42s) 0.8485 Fung / Chinese ✓
45000 45% (1m 54s) 1.7468 Haik / Korean ✗ (Arabic)
50000 50% (2m 7s) 4.2849 Awen / Dutch ✗ (Russian)
55000 55% (2m 19s) 2.6128 Schenck / Czech ✗ (Dutch)
60000 60% (2m 32s) 0.8634 Padovano / Italian ✓
65000 65% (2m 45s) 2.2693 Hussein / Russian ✗ (English)
70000 70% (2m 57s) 2.1921 Guillory / English ✗ (French)
75000 75% (3m 10s) 1.2447 Marszaek / Polish ✓
80000 80% (3m 22s) 0.6800 Totah / Arabic ✓
85000 85% (3m 35s) 1.6248 Rijnder / German ✗ (Dutch)
90000 90% (3m 50s) 0.1394 Ron / Korean ✓
95000 95% (4m 3s) 2.3415 D'cruze / Portuguese ✗ (Spanish)
100000 100% (4m 16s) 2.7241 Smith / Scottish ✗ (Czech

In [58]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [1]:
#  Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    guess, guess_i = categoryFromOutput(output)
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

NameError: name 'torch' is not defined